In [59]:
import pandas as pd
from opentrons import robot, containers, instruments

robot.connect()

def load_map(task,extra_plates=None):
    '''
    Takes in a task and matches it up with the corresponding deck map.
    Can take extra plates in as a list of names and will then pair them
    with the remaining empty slots
    '''
    if task == 'build':
        # For building goldengate reactions
        locations = {
            "tiprack-200" : "A3",
            "tiprack-10" : "E2",
            "tiprack-10s1" : "E3",
            "tiprack-10s2" : "E1",
            "trash" : "D1",
            "PCR-strip-tall" : "C3",
            "DEST_PLATE" : "C2",
            "Tube_rack" : "B1"
        }
        EMPTY_SLOTS = ['D3','D2','B2']
    elif task == 'resuspension':
        # For resuspending plates with lyophilized DNA
        locations = {
            "tiprack-200" : "A3",
            "tiprack-10" : "E2",
            "tiprack-10s1" : "E3",
            "trash" : "D1",
            "PCR-strip-tall" : "C3",
            "PLATE HERE" : "B2",
            "Tube_rack" : "B1"
        }
        EMPTY_SLOTS = ['D3','D2','C2']
    elif task == 'plating':
        locations = {
            "tiprack-200" : "A3",
            "tiprack-10_2" : "E2",
            "tiprack-10_1" : "E3",
            "tiprack-10_3" : "E1",
            "trash" : "D1",
            "PCR-strip-tall" : "C3",
            "Transformation" : "C2",
            "Tube_rack" : "B1"
        }
        EMPTY_SLOTS = ['D3','D2','B2']
    else:
        print("Unknown task, defaulting to standard layout")
        locations = {
            "tiprack-200" : "A3",
            "tiprack-10" : "E2",
            "tiprack-10s1" : "E3",
            "trash" : "D1",
            "PCR-strip-tall" : "C3",
            "Tube_rack" : "B1"
        }
        EMPTY_SLOTS = ['D3','D2','C2','B2']
        print("Available slots: ",EMPTY_SLOTS)
    # Place extra plates in the slots that are currently empty
    if extra_plates != None:
        if len(extra_plates) > len(EMPTY_SLOTS):
            print("Too many extra plates, only using the first {} plates".format(len(EMPTY_SLOTS)))
            print("Will use: {}".format(extra_plates[:len(EMPTY_SLOTS)]))
        locations.update(zip(extra_plates,EMPTY_SLOTS[:len(extra_plates)]))
    # Generate a dataframe that resembles the OT-One deck
    deck = ['A1','B2','C3','D2','E1']
    slots = pd.Series(deck)
    columns = sorted(slots.str[0].unique())
    rows = sorted(slots.str[1].unique(), reverse=True)
    layout_table = pd.DataFrame(index=rows, columns=columns)
    layout_table.fillna("---", inplace=True)

    # Fill in the dataframe with the locations
    for obj in locations:
        layout_table.loc[locations[obj][1], locations[obj][0]] = obj

    # Displays the required plate map and waits to proceed
    print("\n Please arrange the plates in the following configuration: \n")
    print(layout_table,"\n")
    print()
    input("Press 'Enter' to continue")

    return locations

def load_ot(locations):
    container_dict = {
        "tiprack-200" : "tiprack-200ul",
        "tiprack-10" : "tiprack-10ul",
        "tiprack-10s1" : "tiprack-10ul",
        "tiprack-10s2" : "tiprack-10ul",
        "trash" : "point",
        "PCR-strip-tall" : "PCR-strip-tall",
        "DEST_PLATE" : "96-PCR-tall",
        "Tube_rack" : "tube-rack-2ml",
        "PLATE HERE" : "96-flat",
        "New_plate" : "96-flat",
        "Transformation" : "96-PCR-tall",
        "Agar_plate" : "96-deep-well"
    }
    
    
    
p200_tipracks = [
    containers.load('tiprack-200ul', "A3"),
]

p10_tipracks = [
    containers.load('tiprack-10ul', "E2"),
]

p10s_tipracks = [
    containers.load('tiprack-10ul', "E1"),
    containers.load('tiprack-10ul', "E3")
]

trash = containers.load('point', "D1", 'holywastedplasticbatman')
# centrifuge_tube = containers.load('tube-rack-2ml',locations[7,1])
# master = containers.load('PCR-strip-tall', locations[5,1])

# dest_plate = containers.load('96-PCR-tall', locations[6,1])


def load_pipettes(set_p10=True,set_p10s=True,set_p200=True):
    if set_p10:
        print("Loaded p10 multi channel")
        global p10
        p10 = instruments.Pipette(
            axis='a',
            max_volume=10,
            min_volume=0.5,
            tip_racks=p10_tipracks,
            trash_container=trash,
            channels=8,
            name='p10-8',
            aspirate_speed=400,
            dispense_speed=800
        )
    if set_p10s:
        print("Loaded p10 single channel")
        global p10s
        p10s = instruments.Pipette(
            axis='a',
            max_volume=10,
            min_volume=0.5,
            tip_racks=p10s_tipracks,
            trash_container=trash,
            channels=1,
            name='p10-8s',
            aspirate_speed=400,
            dispense_speed=800
        )

    if set_p200:
        global p200
        print("Loaded p200 single channel")
        p200 = instruments.Pipette(
            axis='b',
            max_volume=200,
            min_volume=20,
            tip_racks=p200_tipracks,
            trash_container=trash,
            channels=1,
            name='p200-1',
            aspirate_speed=400,
            dispense_speed=800
        )
    return

    


def initialize_ot(task,extra_plates=None):
    '''
    Takes in the desired task and then returns the deck
    map and loads all of the containers and pipettes needed.
    Takes in an optional parameter for specific plates that 
    change names each time. 
    '''
    print(extra_plates)
    print(task)
    
    
extra_plates = ["plate1","plate2","plate3","plate4"]
    
locations = load_map("resuspension", extra_plates)
    
load_pipettes(set_p10=False)

p10.pick_up_tip()

# initialize_ot("build",["plate1","plate2"])
            

Too many extra plates, only using the first 3 plates
Will use: ['plate1', 'plate2', 'plate3']

 Please arrange the plates in the following configuration: 

             A           B               C       D             E
3  tiprack-200         ---  PCR-strip-tall  plate1  tiprack-10s1
2          ---  PLATE HERE          plate3  plate2    tiprack-10
1          ---   Tube_rack             ---   trash           --- 


Press 'Enter' to continue
Loaded p10 single channel
Loaded p200 single channel
